In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    print("Session runs on GOOGLE COLAB")
    !git clone https://github.com/leuchtum/cse-seminar-windprognose.git
    !pip install wandb
    !pip install pandas --upgrade
    sys.path.append('/content/cse-seminar-windprognose/prognose/')
else:
    print("Session runs on LOCAL MACHINE")

Session runs on LOCAL MACHINE


In [2]:
import MLForecast.sources
import MLForecast.preprocessing as prep
import MLForecast.normalize
import MLForecast.networks
import MLForecast.postprocessing
import MLForecast.visualize
import MLForecast.config

from datetime import datetime
import pandas as pd
import wandb
import pprint

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
if not physical_devices and IN_COLAB:
    raise("Switch runtime to GPU runtime")

# Login to W&B
Only required once per computer uptime!

In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: leuchtum (use `wandb login --relogin` to force relogin)


True

# Set parameters

In [ ]:
X_WIDTH = 200
Y_WIDTH = 12
SHIFT = 200
INTERPOLATE = 4
SPLIT = (.05, .3, .65) # TEST, VAL, TRAIN
LABEL_COLUMNS = ["WX_01886", "WY_01886"]
EPOCHS = 10
EARLY_STOP = 2

structure = ["LSTM_x", "DROP_d", "DENSE_y", "OUT"]
network_config = {
    #"LSTM": {
    #    "recurrent_regularizer": "L1_0.0001"
    #},
    "x": 128,
    "y": 64,
    "d": 0.3
}

In [4]:
INTERPOLATE = 4
SPLIT = (.05, .3, .65) # TEST, VAL, TRAIN
LABEL_COLUMNS = ["WX_01886", "WY_01886"]

# SWEEP
sweep_config = MLForecast.config.config

# Define constants for database build
X_WIDTH = sweep_config["parameters"]["x_width"]["value"]
Y_WIDTH = sweep_config["parameters"]["y_width"]["value"]
SHIFT = sweep_config["parameters"]["x_width"]["value"]

# Make Dataset
This may take a while

In [5]:
print("LOAD")
dwd = MLForecast.sources.DWDStationsHourly(["04887", "01886", "02886", "03402"], ["wind", "air_temperature", "pressure", "sun"])
df_dwd = dwd.get_data(drop_before=datetime(2010,1,1))
cal = MLForecast.sources.CalendricalDataHourly(start=datetime(2010,1,1))
df_cal = cal.get_data()

df = pd.concat([df_dwd, df_cal], axis=1)


print("INTERPOLATE AND POL2CART")
df = prep.interpolate_columnwise(df, INTERPOLATE)
df =  prep.pol2cart(df)

print("FIT NORMALIZER")
spezial_borders = {
    "WX": (-1, 1),
    "WY": (-1, 1)
}
exclude = ["CAL"]
norm = MLForecast.normalize.MinMaxNormalizer(spezial_border=spezial_borders, exclude=exclude)
norm.fit(df)

print("MAKE X_DF AND Y_DF")
df_x = norm.normalize(df)
df_y = df[LABEL_COLUMNS]

print("SAMPLE")
x, y =  prep.sample_sequences(df_x, df_y, X_WIDTH, Y_WIDTH, SHIFT)


print("SPLIT")
test, val, train =  prep.split_test_val_train(x, y, SPLIT)

test_np =  prep.df_tuple_to_np(test)
val_np =  prep.df_tuple_to_np(val)
train_np =  prep.df_tuple_to_np(train)

LOAD
CHECK FOR NEW FILES AT https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/
04887:   0%|          | 0/8 [00:00<?, ?it/s]DOWNLOAD FILES
READ IN
03402: 100%|██████████| 6/6 [00:11<00:00,  1.92s/it]
INTERPOLATE AND POL2CART
FIT NORMALIZER
MAKE X_DF AND Y_DF
 12%|█▏        | 626/5289 [00:00<00:00, 6251.02it/s]SAMPLE

100%|██████████| 417/417 [00:00<00:00, 5358.52it/s]

100%|██████████| 176/176 [00:00<00:00, 3794.54it/s]

100%|██████████| 149/149 [00:00<00:00, 4251.63it/s]

100%|██████████| 365/365 [00:00<00:00, 4619.18it/s]

100%|██████████| 364/364 [00:00<00:00, 5387.88it/s]

100%|██████████| 31/31 [00:00<00:00, 5797.89it/s]

100%|██████████| 85/85 [00:00<00:00, 6322.55it/s]

100%|██████████| 427/427 [00:00<00:00, 6145.97it/s]

100%|██████████| 26/26 [00:06<00:00,  3.99it/s]
SPLIT


In [ ]:
MLForecast.visualize.plot_input_label_example(x[30], y[30])

# Init W&B

In [ ]:
database_report =  prep.DataBaseReport(df_x, df_y, x, y, SHIFT, SPLIT).report()

wandb_config = {
    "structure":structure,
    "config":network_config,
    "database": database_report,
    "epochs": EPOCHS,
    "early_stop": EARLY_STOP,
}

wandb.init(
    project='windprognose',
    entity='leuchtum',
    notes="test",
    config=wandb_config,
)

In [6]:
# SWEEP
#sweep_id = wandb.sweep(sweep_config)
sweep_id = "djxy5uvf"

# Build and train model

In [ ]:
my_model = MLForecast.networks.LSTMseq2vec(structure, network_config)
my_model.set_input_shape(train[0][0].shape)
my_model.set_output_shape(train[1][0].shape)
my_model.build()
my_model.compile()

my_model.fit(train_np, val_np, early_stop=EARLY_STOP, epochs=EPOCHS, track_wandb=True)

In [7]:
def sweep_train():
    with wandb.init() as run:
        config = wandb.config
        l2 = config["l2"]
        structure = config["structure"].split("+")
        network_config = {
            "x": config["x"],
            "y": int(config["x"]/2),
            "d": config["d"],
            "LSTM":{
                "recurrent_regularizer": f"L2_{l2}",
                "activity_regularizer": f"L2_{l2}"
            }
        }

        my_model = MLForecast.networks.LSTMseq2vec(structure, network_config)
        my_model.set_input_shape(train[0][0].shape)
        my_model.set_output_shape(train[1][0].shape)
        my_model.build()
        my_model.compile()

        my_model.fit(
             train_np,
             val_np,
             early_stop=config["early_stop"],
             epochs=config["epochs"],
             track_wandb=True
        )

In [8]:
count = 5 # number of runs to execute
wandb.agent(sweep_id, function=sweep_train, count=count)

wandb: Agent Starting Run: kiqxkc3w with config:
wandb: 	d: 0.35
wandb: 	early_stop: 20
wandb: 	epochs: 200
wandb: 	l2: 1e-06
wandb: 	loss: mae
wandb: 	structure: LSTM_x+DROP_d+LSTM_y+DROP_d+DENSE_y+OUT
wandb: 	x: 32
wandb: 	x_width: 96
wandb: 	y_width: 12
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: wandb version 0.10.32 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/200
700/700 [==============================] - 63s 84ms/step - loss: 5.8387 - root_mean_squared_error: 2.4288 - mean_absolute_error: 1.8768 - mean_squared_error: 5.8993 - val_loss: 5.8943 - val_root_mean_squared_error: 2.2489 - val_mean_absolute_error: 1.7550 - val_mean_squared_error: 5.0573
Epoch 2/200
700/700 [==============================] - 55s 79ms/step - loss: 5.8485 - root_mean_squared_error: 2.4282 - mean_absolute_error: 1.8760 - mean_squared_error: 5.8964 - val_loss: 5.8943 - val_root_mean_squared_error: 2.2489 - val_mean_absolute_error: 1.7549 - val_mean_squared_error: 5.0574
Epoch 3/200
700/700 [==============================] - 57s 81ms/step - loss: 5.9118 - root_mean_squared_error: 2.4282 - mean_absolute_error: 1.8760 - mean_squared_error: 5.8963 - val_loss: 5.8943 - val_root_mean_squared_error: 2.2489 - val_mean_absolute_error: 1.7549 - val_mean_squared_error: 5.0574
Epoch 4/200
700/700 [==============================] - 62s 89ms/step - loss: 5.9426 - root_mean_s

epoch,20
loss,5.93071
root_mean_squared_error,2.42821
mean_absolute_error,1.87585
mean_squared_error,5.89622
val_loss,5.89453
val_root_mean_squared_error,2.24886
val_mean_absolute_error,1.75482
val_mean_squared_error,5.05736
_runtime,1179
_timestamp,1624971606


epoch,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
loss,▃▄▆█▃█▂▇▃▇▆▄▁▄▃▆▄▃▅█▇
root_mean_squared_error,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mean_absolute_error,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mean_squared_error,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁▂▂▃▄▅▆▆▇▇▇▇▇████████
val_root_mean_squared_error,▁█▅▂▃▄▅▄▆▄▆▆▅▅▅▅▆▆▅▅▅
val_mean_absolute_error,█▅▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_mean_squared_error,▁█▅▂▃▄▅▄▆▄▅▅▅▅▅▅▆▆▅▅▅
_runtime,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
_timestamp,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▆▇▇▇██


wandb: Agent Starting Run: a8ue6zz4 with config:
wandb: 	d: 0.15
wandb: 	early_stop: 20
wandb: 	epochs: 200
wandb: 	l2: 5e-05
wandb: 	loss: mse
wandb: 	structure: LSTM_x+DROP_d+LSTM_x+DROP_d+DENSE_y+OUT
wandb: 	x: 256
wandb: 	x_width: 96
wandb: 	y_width: 12
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: wandb version 0.10.32 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/200
700/700 [==============================] - 344s 486ms/step - loss: 5.9096 - root_mean_squared_error: 2.4283 - mean_absolute_error: 1.8761 - mean_squared_error: 5.8965 - val_loss: 5.8949 - val_root_mean_squared_error: 2.2489 - val_mean_absolute_error: 1.7549 - val_mean_squared_error: 5.0574
Epoch 2/200
700/700 [==============================] - 353s 504ms/step - loss: 5.8969 - root_mean_squared_error: 2.4282 - mean_absolute_error: 1.8759 - mean_squared_error: 5.8963 - val_loss: 5.8946 - val_root_mean_squared_error: 2.2489 - val_mean_absolute_error: 1.7548 - val_mean_squared_error: 5.0574
Epoch 3/200
700/700 [==============================] - 394s 563ms/step - loss: 5.8938 - root_mean_squared_error: 2.4282 - mean_absolute_error: 1.8759 - mean_squared_error: 5.8962 - val_loss: 5.8946 - val_root_mean_squared_error: 2.2489 - val_mean_absolute_error: 1.7548 - val_mean_squared_error: 5.0574
Epoch 4/200
700/700 [==============================] - 389s 556ms/step - loss: 5.8918 - roo

# Analyse performance

In [ ]:
performance = MLForecast.postprocessing.PerformanceAnalyser(my_model.model, test_np)
mae_all = performance.by_metrics("mae")
mae_single = performance.by_metrics("mae", by_individual_column=True)

performance_report = performance.report()

suptitle = "Model-Performance für Station 15444"
subtitles = ["Wind in X Richtung", "Wind in Y Richtung"]
note = f"Samples per col: {len(mae_single[0])}; Name of run: {wandb.run.name}"
xtitle = "Prognosezeitpunkt"
ytitle = "Absoluter Fehler"
MLForecast.visualize.boxplot(mae_single, suptitle, subtitles, xtitle, ytitle, note=note)

In [ ]:
pprint.pprint(database_report)